In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
google_api_key = os.getenv("GEMINI_API_KEY")
if google_api_key=="":
    print("not found")
else:
    print("api found")

api found


In [5]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.gemini import Gemini
from IPython.display import Markdown, display
from llama_index.core import ServiceContext
from llama_index.core import StorageContext, load_index_from_storage 
import google.generativeai as genai
from llama_index.embeddings.gemini import GeminiEmbedding

/Users/arsh/Desktop/qasystem/qasystem/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
genai.configure(api_key=google_api_key)

In [7]:
for models in genai.list_models():
    print(models)

Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representatio

In [8]:
for model in genai.list_models():
  if 'generateContent' in model.supported_generation_methods:
    display(model.name)

'models/gemini-1.0-pro-latest'

'models/gemini-1.0-pro'

'models/gemini-pro'

'models/gemini-1.0-pro-001'

'models/gemini-1.0-pro-vision-latest'

...

'models/gemini-1.5-flash-8b'

'models/gemini-1.5-flash-8b-001'

'models/gemini-1.5-flash-8b-latest'

'models/gemini-1.5-flash-8b-exp-0827'

'models/gemini-1.5-flash-8b-exp-0924'

In [9]:
documents = SimpleDirectoryReader("../Data")

In [10]:
data = documents.load_data()

In [11]:
data

[Document(id_='b99283e5-e776-49cd-93fb-238af4f75e2c', embedding=None, metadata={'file_path': '/Users/arsh/Desktop/qasystem/notebook/../Data/sample2.txt', 'file_name': 'sample2.txt', 'file_type': 'text/plain', 'file_size': 12466, 'creation_date': '2024-10-11', 'last_modified_date': '2024-10-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='From Wikipedia, the free encyclopedia\n"AI" redirects here. For other uses, see AI (disambiguation), Artificial intelligence (disambiguation), and Intelligent agent.\nPart of a series on\nArtificial intelligence\n\nMajor goals\nApproaches\nApplications\nPhilosophy\nHistory\nGlossary\nvte\nArtificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is 

In [14]:
print(data[0].text)

From Wikipedia, the free encyclopedia
"AI" redirects here. For other uses, see AI (disambiguation), Artificial intelligence (disambiguation), and Intelligent agent.
Part of a series on
Artificial intelligence

Major goals
Approaches
Applications
Philosophy
History
Glossary
vte
Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.

Some high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, 

In [15]:
gemini_embedding_model = GeminiEmbedding(model_name="models/embedding-001")

In [16]:
# WE CAN CHOOSE MODEL THROUGH GOOGLE GENAI
# model=genai.GenerativeModel("gemini-1.0-pro")
#OR 
# WE CAN USE LLAMA_INDEX's GEMINI MODULE
#model = Gemini(model="gemini-1.0-pro", api_key=google_api_key)

In [18]:
model = Gemini(models="gemini-1.0-pro", api_key=google_api_key)

In [19]:
# SERVICE CONTEXT IS A CONTAINER TO CONTAIN QUERY AND INDEXES AND ALL

In [28]:
from llama_index.core.settings import Settings
from llama_index.core.node_parser import SentenceSplitter

In [29]:
Settings.llm = model
Settings.embed_model = gemini_embedding_model
Settings.node_parser = SentenceSplitter(chunk_size=800, chunk_overlap=20)

In [31]:
index = VectorStoreIndex.from_documents(data,embed_model=gemini_embedding_model,llm=model)

**CREATING LOCAL DATABASE**

In [32]:
index.storage_context.persist()

In [33]:
query_engine = index.as_query_engine()

In [35]:
response = query_engine.query("what is artificial intelligence?")

In [36]:
print(response.response)

Artificial intelligence is the study of intelligent agents, which are systems that can reason, learn, and act autonomously. 



In [37]:
respponse2 = query_engine.query("who is ronaldo ?")

In [39]:
print(respponse2.response)

The provided text does not contain information about Ronaldo. 

